In [ ]:
import os,sys,argparse
import yaml
import numpy as np
sys.path.append("../")
import mlreco
from mlreco.main_funcs import process_config
from mlreco.iotools.factories import loader_factory
import mlreco.iotools.ubparsers
import lardly
from lardly.detectoroutline import DetectorOutline
import chart_studio as cs
import chart_studio.plotly as py
import plotly.graph_objects as go

In [ ]:
cfg_file = "test_ub_loader.yaml"
if not os.path.isfile(cfg_file):
    cfg_file = os.path.join(current_directory, 'config', config)
if not os.path.isfile(cfg_file):
    print(config, 'not found...')
    sys.exit(1)

cfg = yaml.load(open(cfg_file, 'r'), Loader=yaml.Loader)

process_config(cfg)

In [ ]:
# load utility to draw TPC outline
detdata = DetectorOutline()

# define some colors for particle types
ssnet_color_array = np.array( ((0,0,0),      # bg
                               (255,0,0),    # electron
                               (0,255,0),    # gamma
                               (0,0,255),    # muon
                               (255,0,255),  # pion
                               (0,255,255),  # proton
                               (0,0,0)),     # other
                               dtype=np.float64 ) 

# point type: (0=proton,1=(not gamma and not electron and not proton), 2=gamma, 3=shower (primary,nCapture,conv), 4=delta (muIoni,h\Ioni))
keypoint_color_array = np.array(((0,200,200), # proton
                                 (0,0,200),   # track
                                 (0,200,0),   # gamma
                                 (200,0,200),   # shower
                                 (200,200,0)), # delta
                                 dtype=np.float64 )

larcvpid2class = {0:0,#unknown -> bg
                  1:0,#cosmic -> bg
                  2:0,#bnb -> bg
                  3:1,#eminus->electron
                  4:2,#gamma->gamma
                  5:2,#pizero->gamma
                  6:3,#muon->muon
                  7:4,#kaon->other
                  8:4,#pion->pion
                  9:5,#proton->proton
                 }

In [ ]:
# DATA LOADER
data_io = loader_factory(cfg, event_list=None)
def cycle(data_io):
    while True:
        for x in data_io:
            yield x
data_iter = iter(cycle(data_io))

In [ ]:
# Get entry data
batch = next(iter(data_iter))
print(batch.keys())
for name,data in batch.items():
    print(name," : ",type(data)," -------------------")
    if type(data) is np.ndarray:
        print(data.shape," ",data.dtype)
    elif name=="index":
        print("Event INDEX: ",data)
# scale voxel indices to xyz positions in the detector
pos3d = batch["segment_label"][:,1:4].astype(np.float32)
pos3d[:,0:3] *= 0.3 # scale voxels to 0.3 cm
pos3d[:,1] -= 117.0 # center y-axis to y=0

In [ ]:
# process keypoint labels
kpbatch = batch["particles_label"]
print("labels: ",np.unique(kpbatch[:,4]))
kp_data = {}
for ikp in range(5):
    kptype = kpbatch[:,4]
    kppos  = kpbatch[kptype==ikp][:,1:4]*0.3
    kppos[:,1] -= 117.0
    kp_data[ikp] = kppos
    print(" KP[%d] n=%d"%(ikp,kppos.shape[0]))
    #if kppos.shape[0]>0:
    #    print(kppos)


In [ ]:
# Plot Particle
no_ghost_points = True
plot_keypoints = True

labels = batch["segment_label"][:,4]
print(labels.shape)
print(labels[:10])
print("unique labels: ",np.unique(labels))

if no_ghost_points:
    pos = pos3d[labels[:]!=0]
else:
    pos = pos3d
print("pos: ",pos.shape)

plot_labels = labels[labels[:]!=0]
print("plot_labels: ",plot_labels.shape)
labelcol = ssnet_color_array[plot_labels[:]]
print("labelcol: ",labelcol.shape)

particle_plot = {
    "type":"scatter3d",
    "x":pos[:,0],
    "y":pos[:,1],
    "z":pos[:,2],
        "mode":"markers",
        "name":"particle",
        "marker":{"color":labelcol,"size":1}
    }

kp_plot = []
if plot_keypoints:
    for k,kppos in kp_data.items():
        kpcolor = keypoint_color_array[k]
        zcolor = "rgb(%d,%d,%d)"%(kpcolor[0],kpcolor[1],kpcolor[2])
        kp_plotly = {
            "type":"scatter3d",
            "x":kppos[:,0],
            "y":kppos[:,1],
            "z":kppos[:,2],
            "mode":"markers",
            "name":"KP[%d]"%(k),
            "marker":{"color":zcolor,"size":5}
        }
        kp_plot.append(kp_plotly)

detlines = detdata.getlines(color=(10,10,10))

# DATA
particle_plot_data = [particle_plot] + detdata.getlines(color=(10,10,10))
if len(kp_plot)>0:
    particle_plot_data += kp_plot

# LAYOUT
axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(100, 100, 100,0.5)",
    "gridcolor": "rgb(50, 50, 50)",
    "zerolinecolor": "rgb(0, 0, 0)",
}

ave3d = [0.0,0.0,0.0]


layout = go.Layout(
    title='SINGLE PARTICLE VOXELS',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": ave3d[0]+0.1, "y": ave3d[1], "z": ave3d[2]},
                   "center":dict(x=ave3d[0], y=ave3d[1], z=ave3d[2]),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=particle_plot_data, layout=layout)
fig.show()

In [ ]:
# NOTES: ways to improve the data
# photon keypoint location looks off
# where is the neutrino interaction?